# Flexibly Structured Data

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

import requests
from pymongo import MongoClient

client = MongoClient()
db = client['nobel']

## Count of prizes versus laureates

We retrieved two sets of data from the Nobel Prize API. We saved these data to a nobel database as two collections, prizes and laureates. Given a connected client object, client, count the number of documents in the prizes and laureates collections, and pick one of the following statements as true.

Recall that you can access databases by name as attributes of the client (client.<db_name>), collections by name as attributes of databases (<db>.<collection_name>), and that the count_documents method of a collection requires a (perhaps-empty, i.e. {}) filter argument.

## Listing databases and collections

Our MongoClient object is not actually a dictionary, so we can't call keys() to list the names of accessible databases. The same is true for listing collections of a database. Instead, we can list database names by calling list_database_names() on a client instance, and we can list collection names by calling list_collection_names() on a database instance.

In [5]:
# Save a list of names of the databases managed by client
db_names = client.list_database_names()
print(db_names)

# Save a list of names of the collections managed by the "nobel" database
nobel_coll_names = client.nobel.list_collection_names()
print(nobel_coll_names)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## List fields and count laureates' prizes

Use a collection’s find_one method to return a document. This method takes an optional filter argument. Passing an empty filter ({}) is the same as passing no filter. In Python, the returned document takes the form of a dictionary. The keys of the dictionary are the (root-level) "fields" of the document.

Each laureate document has a "prizes" field. This field, an array, stores info about each of the laureate’s (possibly shared) prizes. You may iterate over the collection, collecting from each document. However, a collection is not a list, so we can't write for doc in <collection> to iterate over documents. Instead, call find() to produce an iterable called a cursor, and write for doc in <collection>.find() to iterate over documents.

In [6]:
# Connect to the "nobel" database
db = client.nobel

## 'born' approximation

The born field in a laureate collection document records the date of birth of that laureate. born values are of the form "YYYY-MM-DD", also known as ISO 8601 format. An example value is "1937-02-01", for February 1st, 1937. This format is convenient for lexicographic comparison. For example, the query

    db.laureates.count_documents({"born": {"$lt": "1900"}})

returns the number of laureates with recorded dates of birth earlier than the year 1900. Using the query format above, what is the number of laureates born prior to 1800? What about prior to 1700?

## Composing filters

It is often useful to incrementally build up a filter document in order to see the effect of adding constraints one at a time. In this exercise, we will count the number of laureate documents matching some criteria, and we will gradually add criteria.

In [7]:
# Create a filter for laureates who died in the USA
criteria = {"diedCountry": "USA"}

# Save a count of these laureates
count = db.laureates.count_documents(criteria)
print(count)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## We've got options

Sometimes, we wish to find documents where a field's value matches any of a set of options. We saw that the $in query operator can be used for this purpose. For example, how many laureates were born in any of "Canada", "Mexico", or "USA"?

If we wish to accept all but one option as a value for a field, we can use the $ne (not equal) operator. For example, how many laureates died in the USA but were not born in the USA?

In [ ]:
# Save a filter for laureates born in the USA, Canada, or Mexico
criteria = {"bornCountry": {"$in": ["USA", "Canada", "Mexico"]}}

# Count them and save the count
count = db.laureates.count_documents(criteria)
print(count)

## Starting our ascent

Throughout this course, we will gradually build up a set of tools to examine the proportion of Nobel prizes that were awarded to immigrants. In this exercise, you will answer a limited but related question using tools we have introduced so far.

We saw from his laureate document that Walter Kohn's country of birth was "Austria" and that his prize affiliation country was "USA". Count the number of laureates born in Austria with a prize affiliation country that is not also Austria.

In [8]:
# Filter for laureates born in Austria with non-Austria prize affiliation
criteria = {"bornCountry": "Austria", "prizes.affiliations.country": {"$ne": "Austria"}}

# Count the number of such laureates
count = db.laureates.count_documents(criteria)
print(count)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Our 'born' approximation, and a special laureate

We saw earlier that the laureates collection encodes uncertainty about birthdate in a special way. When a birthdate is unknown, the "born" field has the value "0000-00-00". Thus,

    db.laureates.count_documents({"born": "0000-00-00"})

counts the number of such laureates. Or does it?

We also saw that the total number of laureate prizes is more than the number of laureates -- some were awarded more than one prize. There is one in particular with a whopping three prizes, and this laureate holds key information to aid our quest to determine the proportion of prizes awarded to immigrants.

In [ ]:
# Filter for documents without a "born" field
criteria = {"born": {"$exists": False}}
assert db.laureates.count_documents(criteria) == 0

# Working with Distinct Values and Sets

## Categorical data validation

What expression asserts that the distinct Nobel Prize categories catalogued by the "prizes" collection are the same as those catalogued by the "laureates"? Remember to explore example documents in the console via e.g. db.prizes.find_one() and db.laureates.find_one().

## Never from there, but sometimes there at last

There are some recorded countries of death ("diedCountry") that do not appear as a country of birth ("bornCountry") for laureates. One such country is "East Germany".

In [9]:
# Countries recorded as countries of death but not as countries of birth
countries = set(db.laureates.distinct("diedCountry")) - set(db.laureates.distinct("bornCountry"))
print(countries)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Countries of affiliation

We saw in the last exercise that countries can be associated with a laureate as their country of birth and as their country of death. For each prize a laureate received, they may also have been affiliated with an institution at the time, located in a country.

In [ ]:
# The number of distinct countries of laureate affiliation for prizes
count = len(db.laureates.distinct("prizes.affiliations.country"))
print(count)

## Born here, went there

In which countries have USA-born laureates had affiliations for their prizes?

## Triple plays (mostly) all around

Prizes can be shared, even by more than two laureates. In fact, all prize categories but one – literature – have had prizes shared by three or more laureates.

In [10]:
# Save a filter for prize documents with three or more laureates
criteria = {"laureates.2": {"$exists": True}}

# Save the set of distinct prize categories in documents satisfying the criteria
triple_play_categories = set(db.prizes.distinct("category", criteria))
assert set(db.prizes.distinct("category")) - triple_play_categories == {"literature"}

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Sharing in physics after World War II

What is the approximate ratio of the number of laureates who won an unshared ({"share": "1"}) prize in physics after World War II ({"year": {"$gte": "1945"}}) to the number of laureates who won a shared prize in physics after World War II?

## Meanwhile, in other categories...

We learned in the last exercise that there has been significantly more sharing of physics prizes since World War II: the ratio of the number of laureates who won an unshared prize in physics in or after 1945 to the number of laureates who shared a prize in physics in or after 1945 is approximately 0.13. What is this ratio for prize categories other than physics, chemistry, and medicine?

In [ ]:
# Save a filter for laureates with unshared prizes
unshared = {
    "prizes": {"$elemMatch": {
        "category": {"$nin": ["physics", "chemistry", "medicine"]},
        "share": "1",
        "year": {"$gte": "1945"},
    }}}

# Save a filter for laureates with shared prizes
shared = {
    "prizes": {"$elemMatch": {
        "category": {"$nin": ["physics", "chemistry", "medicine"]},
        "share": {"$ne": "1"},
        "year": {"$gte": "1945"},
    }}}

ratio = db.laureates.count_documents(unshared) / db.laureates.count_documents(shared)
print(ratio)

## Organizations and prizes over time

How many organizations won prizes before 1945 versus in or after 1945?

In [11]:
# Save a filter for organization laureates with prizes won before 1945
before = {
    "gender": "org",
    "prizes.year": {"$lt": "1945"},
    }

# Save a filter for organization laureates with prizes won in or after 1945
in_or_after = {
    "gender": "org",
    "prizes.year": {"$gte": "1945"},
    }

n_before = db.laureates.count_documents(before)
n_in_or_after = db.laureates.count_documents(in_or_after)
ratio = n_in_or_after / (n_in_or_after + n_before)
print(ratio)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Glenn, George, and others in the G.B. crew

There are two laureates with Berkeley, California as a prize affiliation city that have the initials G.S. - Glenn Seaborg and George Smoot. How many laureates in total have a first name beginning with "G" and a surname beginning with "S"?

Evaluate the expression

db.laureates.count_documents({"firstname": Regex(____), "surname": Regex(____)})

in the console, filling in the blanks appropriately.

## Germany, then and now

Just as we saw with Poland, there are laureates who were born somewhere that was in Germany at the time but is now not, and others born somewhere that was not in Germany at the time but now is.

In [ ]:
from bson.regex import Regex

# Filter for laureates with "Germany" in their "bornCountry" value
criteria = {"bornCountry": Regex("Germany")}
print(set(db.laureates.distinct("bornCountry", criteria)))

## The prized transistor

Three people shared a Nobel prize "for their researches on semiconductors and their discovery of the transistor effect". We can filter on "transistor" as a substring of a laureate's "prizes.motivation" field value to find these laureates.

In [12]:
from bson.regex import Regex

# Save a filter for laureates with prize motivation values containing "transistor" as a substring
criteria = {"prizes.motivation": Regex("transistor")}

# Save the field names corresponding to a laureate's first name and last name
first, last = "firstname", "surname"
print([(laureate[first], laureate[last]) for laureate in db.laureates.find(criteria)])

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

# Get Only What You Need, and Fast

## Shares of the 1903 Prize in Physics

You want to examine the laureates of the 1903 prize in physics and how they split the prize. Here is a query without projection:

    db.laureates.find({"prizes": {"$elemMatch": {"category": "physics", "year": "1903"}}})

Which projection(s) will fetch the laureates' full names and prize share info? I encourage you to experiment with the console and re-familiarize yourself with the structure of laureate collection documents.

## Rounding up the G.S. crew

We can use the regular expression operator $regex to find laureates whose initials are G.S. Let's use projection and list comprehension to collect the full names of these laureates by concatenating their first ("firstname") and last ("surname") names.

In [13]:
# Collect a list of full names
names = [" ".join([doc["firstname"], doc["surname"]])
         for doc in db.laureates.find(
             {"firstname": {"$regex": "^G"},
              "surname": {"$regex": "^S"}},
             ["firstname", "surname"])]
print(names)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Doing our share of data validation

In our Nobel prizes collection, each document has an array of laureate subdocuments, each containing information such as the prize share for a laureate. Each "laureates.share" value appears to be the reciprocal of a laureate's fractional share of that prize, encoded as a string. For example, a laureate "share" of "4" means that the laureate received a 1/4 share of the prize.

In [ ]:
from fractions import Fraction

# Save documents, projecting out laureate shares
docs = list(db.prizes.find({}, ["laureates.share"]))

# Confirm that fractional laureate shares sum to 1
check = all(
  1 == sum(Fraction(1, int(laureate["share"]))
           for laureate in doc["laureates"])
  for doc in docs
)
assert check

## What the sort?

This block prints out the first five projections of a sorted query. What "sort" argument fills the blank?

    docs = list(db.laureates.find(
        {"born": {"$gte": "1900"}, "prizes.year": {"$gte": "1954"}},
        {"born": 1, "prizes.year": 1, "_id": 0},
        sort=____))
    for doc in docs[:5]:
        print(doc)

    {'born': '1916-08-25', 'prizes': [{'year': '1954'}]}
    {'born': '1915-06-15', 'prizes': [{'year': '1954'}]}
    {'born': '1901-02-28', 'prizes': [{'year': '1954'}, {'year': '1962'}]}
    {'born': '1913-07-12', 'prizes': [{'year': '1955'}]}
    {'born': '1911-01-26', 'prizes': [{'year': '1955'}]}


## Sorting together: MongoDB + Python

Print out the names of all physics laureates. Print one line for each award year, in chronological order. Each line should list laureates for that year in alphabetical order by surname.

I encourage you to print intermediate results and understand the nested structure of prize documents.

In [14]:
from operator import itemgetter

# Sort by ascending year
sort_spec = [("year", 1)]

## Gap years

The prize in economics was not added until 1969. There have also been many years for which prizes in one or more of the original categories were not awarded. Sorting first by reverse chronological order and second by alphabetical order of category, collect and format prize documents to produce one formatted entry per year listing categories missing for that year.

In [15]:
import itertools
from operator import itemgetter

# Save the set of prize categories awarded in 1901
original_categories = set(db.prizes.distinct("category", {"year": "1901"}))

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## High-share categories

In the year 3030, everybody wants to be a Nobel laureate. Over the last thousand years, many new categories have been added. You serve a MongoDB prizes collection with the same schema as we've seen. Many people theorize that they have a better chance in "high-share" categories. They are hitting your server with similar, long-running queries. It's time to cover those queries with an index.

Which of the following indexes is best suited to speeding up the operation db.prizes.distinct("category", {"laureates.share": {"$gt": "3"}})?

## Recently single?

A prize might be awarded to a single laureate or to several. For each prize category, report the most recent year that a single laureate -- rather than several -- received a prize in that category.

In [ ]:
# Specify an index model for compound sorting
index_model = [("category", 1), ("year", -1)]
db.prizes.create_index(index_model)

# Collect the last single-laureate year for each category
report = ""
for category in sorted(db.prizes.distinct("category")):
    doc = db.prizes.find_one({"category": category, "laureates.share": "1"}, sort=[("year", -1)])
    report += "{category}: {year}\n".format(**doc)

print(report)

## Born and affiliated

Some countries are, for one or more laureates, both their country of birth ("bornCountry") and a country of affiliation for one or more of their prizes ("prizes.affiliations.country"). Find the five countries of birth with the highest counts of such laureates.

In [16]:
from collections import Counter

# Ensure an index on country of birth
db.laureates.create_index([("bornCountry", 1)])

# Collect a count of laureates for each country of birth
n_born_and_affiliated = {
    country: db.laureates.count_documents({"bornCountry": country, "prizes.affiliations.country": country})
    for country in db.laureates.distinct("bornCountry")
}

five_most_common = Counter(n_born_and_affiliated).most_common(5)
print(five_most_common)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Setting a new limit?

How many documents does the following expression return?

    list(db.prizes.find({"category": "economics"},
                        {"year": 1, "_id": 0})
         .sort("year")
         .limit(3)
         .limit(5))


## The first five prizes with quarter shares

Find the first five prizes with one or more laureates sharing 1/4 of the prize. Project our prize category, year, and laureates' motivations.

In [17]:
from pprint import pprint

# Fetch prizes with quarter-share laureate(s)
filter_ = {'laureates.share': '4'}

# Save the list of field names
projection = ['category', 'year', 'laureates.motivation']

# Save a cursor to yield the first five prizes
cursor = db.prizes.find(filter_, projection).sort("year").limit(5)
pprint(list(cursor))

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Pages of particle-prized people

You and a friend want to set up a website that gives information on Nobel laureates with awards relating to particle phenomena. You want to present these laureates one page at a time, with three laureates per page. You decide to order the laureates chronologically by award year. When there is a "tie" in ordering (i.e. two laureates were awarded prizes in the same year), you want to order them alphabetically by surname.

In [ ]:
from pprint import pprint

# Write a function to retrieve a page of data
def get_particle_laureates(page_number=1, page_size=3):
    if page_number < 1 or not isinstance(page_number, int):
        raise ValueError("Pages are natural numbers (starting from 1).")
    particle_laureates = list(
        	db.laureates.find(
                {"prizes.motivation": {"$regex": "particle"}},
                ["firstname", "surname", "prizes"])
            .sort([("prizes.year", 1), ("surname", 1)])
            .skip(page_size * (page_number - 1))
            .limit(page_size))
    return particle_laureates

# Collect and save the first nine pages
pages = [get_particle_laureates(page) for page in range(1,9)]
pprint(pages[0])

# Aggregation Pipelines: Let the Server Do It For You

## Sequencing stages

Here is a cursor, followed by four aggregation pipeline stages:

    cursor = (db.laureates.find(
        projection={"firstname": 1, "prizes.year": 1, "_id": 0},
        filter={"gender": "org"})
     .limit(3).sort("prizes.year", -1))

    project_stage = {"$project": {"firstname": 1, "prizes.year": 1, "_id": 0}}
    match_stage = {"$match": {"gender": "org"}}
    limit_stage = {"$limit": 3}
    sort_stage = {"$sort": {"prizes.year": -1}}

What sequence pipeline of the above four stages can produce a cursor db.laureates.aggregate(pipeline) equivalent to cursor above?

## Aggregating a few individuals' country data

The following query cursor yields birth-country and prize-affiliation-country information for three non-organization laureates:

    cursor = (db.laureates.find(
        {"gender": {"$ne": "org"}},
        ["bornCountry", "prizes.affiliations.country"]
    ).limit(3))


In [18]:
# Translate cursor to aggregation pipeline
pipeline = [
    {"$match": {"gender": {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$limit": 3}  
]

for doc in db.laureates.aggregate(pipeline):
    print("{bornCountry}: {prizes}".format(**doc))

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Passing the aggregation baton to Python

Construct an aggregation pipeline to collect, in reverse chronological order (i.e., descending year), prize documents for all original categories (that is, $in categories awarded in 1901). Project only the prize year and category (including document _id is fine).

The aggregation cursor will be fed to Python's itertools.groupby function to group prizes by year. For each year that at least one of the original prize categories was missing, a line with all missing categories for that year will be printed.

In [ ]:
from collections import OrderedDict
from itertools import groupby
from operator import itemgetter

original_categories = set(db.prizes.distinct("category", {"year": "1901"}))

# Save an pipeline to collect original-category prizes
pipeline = [
    {"$match": {"category": {"$in": sorted(original_categories)}}},
    {"$project": {"category": 1, "year": 1}},
    {"$sort": OrderedDict([("year", -1)])}
]
cursor = db.prizes.aggregate(pipeline)
for key, group in groupby(cursor, key=itemgetter("year")):
    missing = original_categories - {doc["category"] for doc in group}
    if missing:
        print("{year}: {missing}".format(year=key, missing=", ".join(sorted(missing))))

## Field Paths and Sets

Previously, we confirmed -- via a Python loop -- that for each prize, either all laureates have a 1/3 share, or none do. Now, let's do this via an aggregation (result should be an empty list):

    list(db.prizes.aggregate([
        {"$project": {"allThree": {"$setEquals": [____, ____]},
                      "noneThree": {"$not": {"$setIsSubset": [____, ____]}}}},
        {"$match": {"$nor": [{"allThree": True}, {"noneThree": True}]}}]))

Which values fill the blanks?

## Organizing prizes

In the slides at the beginning of this lesson, we saw a two-stage aggregation pipeline to determine the number of prizes awarded in total. How many prizes were awarded (at least partly) to organizations?

In [19]:
# Count prizes awarded (at least partly) to organizations as a sum over sizes of "prizes" arrays.
pipeline = [
    {"$match": {"gender": "org"}},
    {"$project": {"n_prizes": {"$size": "$prizes"}}},
    {"$group": {"_id": None, "n_prizes_total": {"$sum": "$n_prizes"}}}
]

print(list(db.laureates.aggregate(pipeline)))

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Gap years, aggregated

In a previous exercise, you collected instances of prize categories not being awarded in particular years. You implemented this using a for loop in Python. You will now implement this as an aggregation pipeline that:

    Filters for original prize categories (i.e. sans economics),
    Projects category and year,
    Groups distinct prize categories awarded by year,
    Projects prize categories not awarded by year,
    Filters for years with missing prize categories, and
    Returns a cursor of documents in reverse chronological order, one per year, each with a list of missing prize categories for that year.

Remember to use field paths (prepend field names with "$") to extract field values in expressions.

In [ ]:
from collections import OrderedDict

original_categories = sorted(set(db.prizes.distinct("category", {"year": "1901"})))
pipeline = [
    {"$match": {"category": {"$in": original_categories}}},
    {"$project": {"category": 1, "year": 1}},
    
    # Collect the set of category values for each prize year.
    {"$group": {"_id": "$year", "categories": {"$addToSet": "$category"}}},
    
    # Project categories *not* awarded (i.e., that are missing this year).
    {"$project": {"missing": {"$setDifference": [original_categories, "$categories"]}}},
    
    # Only include years with missing categories
    {"$match": {"missing.0": {"$exists": True}}},
    
    # Sort in reverse chronological order. Note that "_id" is a distinct year at this stage.
    {"$sort": OrderedDict([("_id", -1)])},
]
for doc in db.prizes.aggregate(pipeline):
    print("{year}: {missing}".format(year=doc["_id"],missing=", ".join(sorted(doc["missing"]))))

## Embedding aggregation expressions

The $expr operator allows embedding of aggregation expressions in a normal query (or in a $match stage). Which of the following expressions counts the number of laureate documents with string-valued bornCountries when passed to db.laureates.count_documents?

You can assume (and check!) that the following is true:

    assert all(isinstance(v, str) for v in db.laureates.distinct("bornCountry"))


## Here and elsewhere

What proportion of laureates won a prize while affiliated with an institution in their country of birth? Build an aggregation pipeline to get the count of laureates who either did or did not win a prize with an affiliation country that is a substring of their country of birth -- for example, the prize affiliation country "Germany" should match the country of birth "Prussia (now Germany)".

In [20]:
pipeline = [
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
  
    # Ensure a single prize affiliation country per pipeline document
    {"$unwind": "$prizes"},
    {"$unwind": "$prizes.affiliations"},
  
    # Filter out "empty" prize-affiliation-country values
    {"$match": {"prizes.affiliations.country": {"$in": db.laureates.distinct("prizes.affiliations.country")}}},
    {"$project": {"affilCountrySameAsBorn": {
        "$gte": [{"$indexOfBytes": ["$prizes.affiliations.country", "$bornCountry"]}, 0]}}},
  
    # Count documents grouped by value (either True or False) for "$affilCountrySameAsBorn"
    {"$group": {"_id": "$affilCountrySameAsBorn", "count": {"$sum": 1}}},
]
for doc in db.laureates.aggregate(pipeline): print(doc)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Countries of birth by prize category

Some prize categories have laureates hailing from a greater number of countries than do other categories. You will build an aggregation pipeline for the prizes collection to collect these numbers, using a $lookup stage to obtain laureate countries of birth.

In [ ]:
pipeline = [
    # Unwind the laureates array
    {"$unwind": "$laureates"},
    {"$lookup": {"from": "laureates", "foreignField": "id",
                 "localField": "laureates.id", "as": "laureate_bios"}},
  
    # Unwind the new laureate_bios array
    {"$unwind": "$laureate_bios"},
    {"$project": {"category": 1, "bornCountry": "$laureate_bios.bornCountry"}},
  
    # Collect bornCountry values associated with each prize category
    {"$group": {"_id": "$category", "bornCountries": {"$addToSet": "$bornCountry"}}},
  
    # Project out the size of each category's (set of) bornCountries
    {"$project": {"category": 1, "nBornCountries": {"$size": "$bornCountries"}}},
    {"$sort": {"nBornCountries": -1}},
]
for doc in db.prizes.aggregate(pipeline): print(doc)

## "...it's the life in your years"

For the pipeline we developed in the last slide deck, I want you to replace the last ($bucket) stage with one such that, given the documents docs collected, we can get the following output:

    from operator import itemgetter

    print(max(docs, key=itemgetter("years")))
    print(min(docs, key=itemgetter("years")))

    {'firstname': 'Rita', 'surname': 'Levi-Montalcini', 'years': 103.0}
    {'firstname': 'Martin Luther', 'surname': 'King Jr.', 'years': 39.0}

You may assume that any earlier $project stage has been replaced by an equivalent $addFields stage.

## How many prizes were awarded to immigrants?

How many prizes were awarded to people who had no affiliation in their country of birth at the time of the award?

In [21]:
pipeline = [
    # Limit results to people; project needed fields; unwind prizes
    {"$match": {"gender": {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$unwind": "$prizes"},
  
    # Count prizes with no country-of-birth affiliation
    {"$addFields": {"bornCountryInAffiliations": {"$in": ["$bornCountry", "$prizes.affiliations.country"]}}},
    {"$match": {"bornCountryInAffiliations": False}},
    {"$count": "awardedElsewhere"},
]

print(list(db.laureates.aggregate(pipeline)))

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

## Refinement: filter out "unaffiliated" people

In the previous exercise, we counted prizes awarded to people without an affiliation in their "bornCountry". However, hundreds of prizes were awarded to people without recorded affiliations; sure, their "bornCountry" is technically not the "country" of any of their affiliations, but there are no "country" values to compare against!

In [ ]:
pipeline = [
    {"$match": {"gender": {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$unwind": "$prizes"},
    {"$addFields": {"bornCountryInAffiliations": {"$in": ["$bornCountry", "$prizes.affiliations.country"]}}},
    {"$match": {"bornCountryInAffiliations": False}},
    {"$count": "awardedElsewhere"},
]

# Construct the additional filter stage
added_stage = {"$match": {"prizes.affiliations.country": {"$in": db.laureates.distinct("prizes.affiliations.country")}}}

# Insert this stage into the pipeline
pipeline.insert(3, added_stage)
print(list(db.laureates.aggregate(pipeline)))